# 🧠 欢迎来到未来科技营

请确保你的设备已正确连接所有部件，并准备好体验互动吧！🚀

## 初始化

In [ ]:
# 📦 导入库
import os
import time
import alsaaudio
import sounddevice as sd
import simpleaudio as sa

from time import sleep

# 配置音频
# 设置 ALSA_CARD 为第一个 USB 输出设备的索引（如存在）
# 查找包含 'usb' 的音频输出设备
dev = next((d for d in sd.query_devices() if d['max_output_channels'] > 0 and 'usb' in d['name'].lower()), None)
# 设置 ALSA_CARD 环境变量以使用 USB 设备
if dev: os.environ['ALSA_CARD'] = str(dev['index'])

## 反馈系统

### 反馈系统 - 大灯

In [2]:
# 初始化 IO，仅能执行一次
from gpiozero import LED
led = LED(24)

In [9]:
# 开灯一秒，然后关灯
led.on()
sleep(1)
led.off()

### 反馈系统 - 声音

In [5]:
# 设置音量，建议每次播放前都调整音量
m = alsaaudio.Mixer('PCM')
# 设置音量为 60%
m.setvolume(60)

# 播放测试音频
# 加载并播放音频文件
wave_obj = sa.WaveObject.from_wave_file("media/test.wav")
play_obj = wave_obj.play()
play_obj.wait_done()

### 反馈系统 - 转动

In [8]:
# 初始化代码，仅需执行一次

# 📦 导入库
from lib.DFRobot_RaspberryPi_DC_Motor import DFRobot_DC_Motor_IIC as Board
board = Board(1, 0x10)    # RaspberryPi select bus 1, set address to 0x10

In [13]:
if board.begin() != board.STA_OK:    # 初始化开发板并检查状态是否正常
    print("board begin faild")       # 如果初始化失败，打印失败信息
else:
    print("board begin success")     # 初始化成功，打印成功信息

    board.set_encoder_enable(board.ALL)                 # 启用所有电机的编码器
    # board.set_encoder_disable(board.ALL)              # （可选）禁用所有电机的编码器
    board.set_encoder_reduction_ratio(board.ALL, 43)    # 设置所有电机的编码器减速比（测试电机减速比为43.8）

    board.set_moter_pwm_frequency(1000)   # 设置电机的PWM频率为1000Hz

    for duty in range(0, 100, 20):   # 从低占空比到高，占空比每次增加10
        board.motor_movement([board.M1], board.CW, duty)    # 电机1正转（顺时针），设置占空比
        board.motor_movement([board.M2], board.CCW, duty)   # 电机2反转（逆时针），设置占空比
        time.sleep(0.5)                                       # 等待1秒
        speed = board.get_encoder_speed(board.ALL)         # 获取所有编码器的转速
        print("duty: %d, M1 encoder speed: %d rpm, M2 encoder speed %d rpm" % (duty, speed[0], speed[1]))  # 打印转速信息

    #for duty in range(100, 0, -20):   # 从高占空比到低，占空比每次减少10
     #   board.motor_movement([board.M1], board.CCW, duty)    # 电机1反转（逆时针），设置占空比
      #  board.motor_movement([board.M2], board.CW, duty)   # 电机2正转（顺时针），设置占空比
     #   time.sleep(0.5)                                       # 等待1秒
     #   speed = board.get_encoder_speed(board.ALL)         # 获取所有编码器的转速
     #   print("duty: %d, M1 encoder speed: %d rpm, M2 encoder speed %d rpm" % (duty, speed[0], speed[1]))  # 打印转速信息
   

board begin success
duty: 0, M1 encoder speed: 0 rpm, M2 encoder speed 0 rpm
duty: 20, M1 encoder speed: 19 rpm, M2 encoder speed -14 rpm
duty: 40, M1 encoder speed: 58 rpm, M2 encoder speed -33 rpm
duty: 60, M1 encoder speed: 100 rpm, M2 encoder speed -67 rpm
duty: 80, M1 encoder speed: 158 rpm, M2 encoder speed -149 rpm


In [14]:
board.motor_stop(board.ALL)   # stop all DC motor

### 反馈系统 - 显示图片

In [11]:
# 初始化代码，仅能执行一次

from lib import LCD_1inch28
from PIL import Image

disp = LCD_1inch28.LCD_1inch28()
disp.Init() # 点亮屏幕

# Clear display.
disp.clear()
#Set the backlight to 50
disp.bl_DutyCycle(50)

In [12]:
# 读取并打开图像文件
image = Image.open('media/test_lcd.jpg')
image = image.rotate(270)

# 显示图像
disp.ShowImage(image) # 显示图像

### 反馈系统 - 显示GIF表情

In [13]:
# 读取并打开图像文件
gif = Image.open('media/test_blink.gif')

# 循环显示 3 次
for i in range(3):
    for i in range(gif.n_frames):
        gif.seek(i)
        image = gif.convert("RGB")
        image = image.rotate(270)
        image = image.resize((240, 240))
        disp.ShowImage(image) # 显示图像

# 感知系统

### 感知系统 - 听觉（麦克风）

In [14]:
# 初始化代码，仅需执行一次

# 📦 导入库
from scipy.io.wavfile import write
import numpy as np

In [15]:
fs = 44100 # 采样频率
seconds = 5 # 录音时长，单位：秒

print("* 开始录音")
myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=1) # 录制音频
sd.wait() # 等待录音完成
print("* 录音结束")

# 将浮点格式转换为整数格式
myrecording = np.int16(myrecording / np.max(np.abs(myrecording)) * 32767)

# 保存音频文件
write("media/test_mic.wav", fs, myrecording)

* 开始录音
* 录音结束


In [16]:
# 播放录制音频文件

# 设置音量
m = alsaaudio.Mixer('PCM')
# 设置音量为 50%
m.setvolume(50)

# 播放测试音频
# 加载并播放音频文件
wave_obj = sa.WaveObject.from_wave_file("media/test_mic.wav")
play_obj = wave_obj.play()
play_obj.wait_done()

### 感知系统 - 超声波测距

In [17]:
# 初始化代码，仅能执行一次

# 导入库
from gpiozero import DistanceSensor
from time import sleep

# 注意配置引脚
sensor = DistanceSensor(echo=6, trigger=26)

/usr/lib/python3/dist-packages/gpiozero/input_devices.py:852: PWMSoftwareFallback: For more accurate readings, use the pigpio pin factory.See https://gpiozero.readthedocs.io/en/stable/api_input.html#distancesensor-hc-sr04 for more info
  warnings.warn(PWMSoftwareFallback(
/usr/lib/python3/dist-packages/gpiozero/input_devices.py:975: DistanceSensorNoEcho: no echo received
  warnings.warn(DistanceSensorNoEcho('no echo received'))


In [20]:
print('Distance: %.1f cm' % (sensor.distance * 100))  # 返回的是 m，需要乘100变成 cm

Distance: 100.0 cm


### 感知系统 - 视觉

In [ ]:
# 📦 导入库
import cv2
from PIL import Image

# 初始化摄像头
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ 摄像头无法打开")
else:
    for _ in range(5):  # 先热身几帧
        cap.read()

    ret, frame = cap.read()
    cap.release()

    if ret:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # image = Image.fromarray(frame_rgb)
        # image = image.resize((320, 240))  # 缩放为 320x240
        # disp.ShowImage(image)  # 显示图像
        cv2.imwrite("media/test_camera.jpg", frame)
        print("✅ 拍照成功，图片保存为 test_camera.jpg")
    else:
        print("❌ 拍照失败")

In [ ]:
from IPython.display import Image as displayImage, display
display(displayImage(filename="media/test_camera.jpg", width=480))

In [4]:
# 读取并打开图像文件
from PIL import Image, ImageOps

image = Image.open("media/test_camera.jpg").rotate(270)
image = ImageOps.fit(image, (240, 240), method=Image.LANCZOS, centering=(0.5, 0.5))
disp.ShowImage(image)  # 显示图像

NameError: name 'disp' is not defined

### 感知系统 - 雷达

In [62]:
from gpiozero import PWMOutputDevice

# 电机速度控制（接 PWMA，即 GPIO13），仅能初始化一次
pwm = PWMOutputDevice(13)

GPIOPinInUse: pin GPIO13 is already in use by <gpiozero.PWMOutputDevice object on pin GPIO13, active_high=True, is_active=True>

In [63]:
# 停止
pwm.off()
'''
# 慢慢加速
for speed in [0.2, 0.4, 0.6, 0.8, 1.0]:
    pwm.value = speed
    print(f"当前速度：{speed}")
    sleep(1)

pwm.value = 0.1'''

'\n# 慢慢加速\nfor speed in [0.2, 0.4, 0.6, 0.8, 1.0]:\n    pwm.value = speed\n    print(f"当前速度：{speed}")\n    sleep(1)\n\npwm.value = 0.1'

In [36]:
import ydlidar
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

# 初始化 Lidar
ydlidar.os_init()
laser = ydlidar.CYdLidar()
laser.setlidaropt(ydlidar.LidarPropSerialPort, "/dev/ttyAMA2")
laser.setlidaropt(ydlidar.LidarPropSerialBaudrate, 115200)
laser.setlidaropt(ydlidar.LidarPropLidarType, ydlidar.TYPE_TRIANGLE)
laser.setlidaropt(ydlidar.LidarPropDeviceType, ydlidar.YDLIDAR_TYPE_SERIAL)
laser.setlidaropt(ydlidar.LidarPropScanFrequency, 4.0)
laser.setlidaropt(ydlidar.LidarPropSampleRate, 3)
laser.setlidaropt(ydlidar.LidarPropSingleChannel, True)

ret = laser.initialize()
if ret:
    ret = laser.turnOn()
    scan = ydlidar.LaserScan()

    try:
        while ret and ydlidar.os_isOk():
            r = laser.doProcessSimple(scan)
            if r:
                points = scan.points
                angles = np.array([p.angle for p in points])
                ranges = np.array([p.range for p in points])

                # 原始坐标（未旋转）
                xs_orig = ranges * np.cos(angles)
                ys_orig = ranges * np.sin(angles)

                # 顺时针旋转 90 度后的坐标
                xs = ys_orig
                ys = -xs_orig

                # 清空前一张图
                clear_output(wait=True)
                plt.figure(figsize=(12, 6))

                # # -------- 笛卡尔坐标图 --------
                # plt.subplot(1, 2, 1)
                # plt.scatter(xs, ys, s=2, c='blue')
                # plt.title("Cartesian View (Rotated 90°)")
                # plt.xlabel("X (m)")
                # plt.ylabel("Y (m)")
                # plt.xlim(-12, 12)
                # plt.ylim(-12, 12)
                # plt.grid(True)
                # plt.gca().set_aspect('equal', adjustable='box')

                # -------- 极坐标图 --------
                ax = plt.subplot(1, 2, 2, projection='polar')
                ax.scatter(angles, ranges, s=2, c='red')
                ax.set_title("Polar View (Raw Lidar Data)")
                ax.set_theta_zero_location('S')  # 正前方朝上
                ax.set_theta_direction(-1)       # 顺时针为正方向

                ax.set_rmax(12)
                ax.grid(True)

                plt.suptitle(f"YDLidar Scan at {scan.stamp}")
                plt.tight_layout()
                plt.show()

            else:
                print("Failed to get Lidar Data")

            time.sleep(0.05)

    except KeyboardInterrupt:
        print("Interrupted by user")

    finally:
        laser.turnOff()
        laser.disconnecting()


Interrupted by user
signal_handler(2)
[2025-07-25 15:43:32][info] Now lidar scanning has stopped!


### 其它 - 温度

In [49]:
def get_cpu_temp():
    with open("/sys/class/thermal/thermal_zone0/temp", "r") as f:
        temp_str = f.readline()
    return float(temp_str) / 1000  # 单位是°C

print(f"CPU 温度: {get_cpu_temp():.2f} °C")

CPU 温度: 53.45 °C


### 其它 - 电量

In [39]:
from lib import INA219
ina219 = INA219.INA219(addr=0x42)

In [48]:
bus_voltage = ina219.getBusVoltage_V()             # 获取负载侧的总线电压（V-端的电压）
shunt_voltage = ina219.getShuntVoltage_mV() / 1000 # 获取分流电阻两端电压（V+ 与 V- 之间），单位换算为伏特
current = ina219.getCurrent_mA()                   # 获取通过分流电阻的电流，单位为毫安（mA）
power = ina219.getPower_W()                        # 获取功率，单位为瓦特（W）

# 将电压映射为一个百分比，通常用于电量或亮度等显示（6V 映射为 0%，8.4V 映射为 100%）
p = (bus_voltage - 6)/2.4*100
if(p > 100): p = 100                               # 超过100%则限制为100%
if(p < 0): p = 0                                   # 小于0%则限制为0%

# INA219 测量的是负载端（V-）的电压，因此电源电压 = bus_voltage + shunt_voltage
# print("PSU Voltage:   {:6.3f} V".format(bus_voltage + shunt_voltage)) # 如需显示电源端电压可取消注释
# print("Shunt Voltage: {:9.6f} V".format(shunt_voltage))              # 如需显示分流电压可取消注释

print("Load Voltage:  {:6.3f} V".format(bus_voltage))           # 打印负载电压
print("Current:       {:9.6f} A".format(current/1000))          # 打印电流（单位换算为安培）
print("Power:         {:6.3f} W".format(power))                 # 打印功率
print("Percent:       {:3.1f}%".format(p))                      # 打印映射后的百分比数值


Load Voltage:   8.156 V
Current:       -0.089900 A
Power:          0.586 W
Percent:       89.8%


## 🎉 恭喜你完成了的所有测试步骤！

下一步你可以尝试：
- 添加语音识别模块
- 使用图像识别进行互动反馈
- ...

继续探索吧！💡

In [46]:
import os
import base64

from openai import OpenAI

# 读取本地图片文件
with open("media/test_camera.jpg", "rb") as f:
    image_data = f.read()
    encoded_image = base64.b64encode(image_data).decode("utf-8")

# 构造 data URL
data_url = f"data:image/jpeg;base64,{encoded_image}"

client = OpenAI(
    base_url="https://ark.cn-beijing.volces.com/api/v3",
    api_key="64876db3-dd82-4f63-8718-2b12eaca3f28",
)
response = client.chat.completions.create(
    model="doubao-1-5-thinking-vision-pro-250428",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": data_url
                    },
                },
                {"type": "text", "text": "描述一下"},
            ],
        }
    ],
)
print(response.choices[0].message.content)

这张图片展示了一个**现代化开放式办公室**的场景：  

- **人员与活动**：画面中有多位员工在工位上使用电脑（可见苹果笔记本）专注工作，着装偏向日常办公风格（如白衬衫、粉色上衣等），呈现出忙碌的工作氛围。  
- **空间与环境**：办公室天花板设计简洁现代，装有嵌入式照明灯、通风口，还有带圆形镂空的装饰板；光线充足（顶部大灯与环境光结合），空间开阔。背景中可见墙面嵌入的大屏幕显示器，进一步强化了办公空间的科技感与协作属性。  

整体环境明亮、规整，传递出**高效、开放**的现代办公气息。
